# Unit Testing Self organising maps using KNN anomaly detection

In [1]:
import pandas as pd
import numpy as np
import json
import os

#importing the dependencies
from anomaly_detectors.som_knn_detector import som_knn_wrapper as som_wrapper
from anomaly_detectors.utils import reader_helper
from anomaly_detectors.utils import csv_prep_for_reader as csv_helper
from anomaly_detectors.reader_writer import db_properties as db_properties
from anomaly_detectors.reader_writer import writer_configs as writer_configs

import warnings
warnings.filterwarnings('ignore')

/home/rohithram/anaconda3/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [2]:
% matplotlib inline
%load_ext autoreload
%autoreload 2

In [3]:
'''
Arguments for reader module to get data from opentsdb
This is included for now just for testing, later the main function will take json as direct input
'''

assetno = ['TSFAD_A1']
con = '52.224.236.31:4242'
src_type =  'opentsdb'

param=['ec2_cpu_utilization_5f5533', 'rds_cpu_utilization_cc0c53']
from_timestamp=1392388200
to_timestamp=1393597320

In [4]:
reader_kwargs= lambda:{
            'assetno':['TSFAD_A1'],
            'from_timestamp':from_timestamp,
            'to_timestamp':to_timestamp,
            'con':con,
            'para_list':param,
            'source_type':src_type,
            'table_name':'',
            'qry_str':'',
            'impute_fill_method':'forward',
            'down_sampling_method':None,
            'down_sampling_window':None,
            'freq':None,
            'resample_fill_method':None,
            'to_resample':None,
            'to_impute':True,
}

model_input_args = lambda :{
    'network_shape':(8,8),
    'input_feature_size':None,
    'time_constant':None,
    'minNumPerBmu':2,
    'no_of_neighbours':3,
    'init_radius':0.4,
    'init_learning_rate':0.01,
    'N':100,    
    'diff_order':1
}

training_args = lambda:{
            'is_train':True,
            'epochs':5,
            'batch_size':4,
            'to_plot':True,
            'test_frac':0.2
        }


        
eval_args = lambda: {
    'model_path':'',
    'to_plot':True,
    'anom_thres':3.0
}

In [5]:
keys = list(model_input_args().keys())+list(training_args().keys())+list(eval_args().keys())

In [6]:
keys

['network_shape',
 'input_feature_size',
 'time_constant',
 'minNumPerBmu',
 'no_of_neighbours',
 'init_radius',
 'init_learning_rate',
 'N',
 'diff_order',
 'is_train',
 'epochs',
 'batch_size',
 'to_plot',
 'test_frac',
 'model_path',
 'to_plot',
 'anom_thres']

## Test for modes :
#### Testing three different modes of the program
* First training and for testing three different modes are tested and the output is shown below

In [7]:
def get_kwargs():
    return model_input_args(),training_args(),eval_args()

In [8]:
reader_kwargs1= reader_kwargs()
json_data = reader_helper.read(reader_kwargs=reader_kwargs1)
model_input_args1,training_args1,eval_args1 = get_kwargs()
training_args1['to_plot'] = False

for i in range(3):
    mode = som_wrapper.mode_options[i]
    print("\nTesting mode option : {}\n".format(mode))
    training_args1['to_plot']=False
    train_res = json.loads(som_wrapper.train(**{**model_input_args1,**training_args1},json_data=json_data))
    eval_args1['model_path'] = train_res['models'][0][assetno[0]]
#     json_data = reader_helper.read(reader_kwargs1)
    eval_args1['to_plot']=False
    test_res = som_wrapper.evaluate(**eval_args1,json_data=json_data,mode=som_wrapper.mode_options[i])
    print(test_res)

http://52.224.236.31:4242/api/query?start=1392388200&end=1393597320&ms=true&m=max:none:ec2_cpu_utilization_5f5533{AssetNo=TSFAD_A1}

Testing mode option : detect only

Data reader initialised 



KeyError: 'models'

## Test 1 : 
#### Testing with parameters being empty quotes

In [ ]:
reader_kwargs1= reader_kwargs()
json_data = reader_helper.read(reader_kwargs=reader_kwargs1)
model_input_args1,training_args1,eval_args1 = get_kwargs()

## Testing on model_input_args

In [ ]:
for key in list(model_input_args1.keys()):
    print("\nGiving {} parameter : ''\n".format(key))
    model_input_args1 = model_input_args()
    model_input_args1[key] = ''
    training_args1['to_plot']=False
    train_res = json.loads(som_wrapper.train(**{**model_input_args1,**training_args1},json_data=json_data))
    print(train_res)

## Testing on training args

In [ ]:
model_input_args1 = model_input_args()

for key in list(training_args().keys()):
    training_args1 = training_args()
    training_args1[key] = ''
#     eval_args1['to_plot']=False
    training_args1['to_plot']=False
    train_res = json.loads(som_wrapper.train(**{**model_input_args1,**training_args1},json_data=json_data))
    print(train_res)

## Testing on evaluation args

In [ ]:
training_args1 = training_args()
model_input_args1 = model_input_args()
training_args1['to_plot']=False
train_res = json.loads(som_wrapper.train(**{**model_input_args1,**training_args1},json_data=json_data))
print(train_res)

In [ ]:
for key in list(eval_args().keys()):
    eval_args1 = eval_args()
    eval_args1['model_path'] = train_res['models'][0][assetno[0]]
    eval_args1[key] = ''
#     json_data = reader_helper.read(reader_kwargs1)
    eval_args1['to_plot']=False
    test_res = som_wrapper.evaluate(**eval_args1,json_data=json_data)
    print(test_res)

## Test 2 :
#### Testing missing parameters : 
* Since only model path is required arg, it doesn't throw any error when we don't pass other evalution args since they are optional

#### Model input args

In [ ]:
for key in list(model_input_args1.keys()):
    print("\nGiving {} parameter : ''\n".format(key))
    model_input_args1 = model_input_args()
    del model_input_args1[key]
    training_args1['to_plot']=False
    train_res = json.loads(som_wrapper.train(**{**model_input_args1,**training_args1},json_data=json_data))
    print(train_res)

#### Training args

In [ ]:
for key in list(training_args1.keys()):
    print("\nGiving {} parameter : ''\n".format(key))
    model_input_args1 = model_input_args()
    del training_args1[key]
#     training_args1['to_plot']=False
    train_res = json.loads(som_wrapper.train(**{**model_input_args1,**training_args1},json_data=json_data))
    print(train_res)

#### Evaluation args

In [ ]:
for key in list(eval_args().keys()):
    eval_args1 = eval_args()
    del eval_args1[key]
    eval_args1['model_path'] = train_res['models'][0][assetno[0]]

#     json_data = reader_helper.read(reader_kwargs1)
    eval_args1['to_plot']=False
    test_res = som_wrapper.evaluate(**eval_args1,json_data=json_data)
    print(test_res)

## Test 3:
#### Testing parameter type mismatch 

### Mismatched params arguments

In [ ]:
train_vals = {'batch_size': 4.5,
             'epochs': 5.5,
             'is_train': 'True',
             'test_frac': 3.0,
             'to_plot': 'True'}

model_vals = {'N': '100',
             'diff_order': 1.4,
             'init_learning_rate': 1,
             'init_radius': 'f',
             'input_feature_size': 'ff',
             'minNumPerBmu': 2,
             'network_shape': ('8', 8),
             'no_of_neighbours': 34.4,
             'time_constant': 'None'}

eval_vals = {
            'model_path':34,
            'to_plot':'True',
            'anom_thres':4
}

####  Testing training args

In [ ]:
for i,key in enumerate(list(training_args().keys())):
    print("\nGiving {} parameter : {}\n".format(key,train_vals[key]))
    model_input_args1 = model_input_args()
    training_args1 = training_args()
    training_args1[key] = train_vals[key]
    train_res = json.loads(som_wrapper.train(**{**model_input_args1,**training_args1},json_data=json_data))
    print(train_res)

####  Testing model input args

In [ ]:
for i,key in enumerate(list(model_input_args().keys())):
    print("\nGiving {} parameter : {}\n".format(key,model_vals[key]))
    training_args1 = training_args()
    model_input_args1 = model_input_args()
    model_input_args1[key] = model_vals[key]
    train_res = json.loads(som_wrapper.train(**{**model_input_args1,**training_args1},json_data=json_data))
    print(train_res)

#### Testing eval args

In [ ]:
training_args1 = training_args()
model_input_args1 = model_input_args()
training_args1['to_plot'] = False

train_res = json.loads(som_wrapper.train(**{**model_input_args1,**training_args1},json_data=json_data))

In [ ]:
eval_vals

In [ ]:
for key in list(eval_args().keys()):
    eval_args1 = eval_args()
    eval_args1[key] = eval_vals[key]

    test_res = som_wrapper.evaluate(**eval_args1,json_data=json_data)
    print(test_res)

### User Defined Test Case:
#### Testing the data missing case

In [ ]:
reader_kwargs1 = reader_kwargs()
reader_kwargs1['from_timestamp'] = int(2**40)

json_data = reader_helper.read(reader_kwargs=reader_kwargs1)

model_input_args1 = model_input_args()
training_args1 = training_args()
training_args1['to_plot']=False
train_res = json.loads(som_wrapper.train(**{**model_input_args1,**training_args1},json_data=json_data))
print(train_res)

## Test 6:
#### Testing Asset Timeline Logging :

## Test 7:
#### Testing the response from program to follow agreed upon template :

In [ ]:
reader_kwargs1 = csv_helper.get_csv_kwargs(infile='../../dataset/lstm_train_data_201B.csv',
                                          filename='lstm_train_data_201B.csv',has_time=False)

json_data = reader_helper.read(reader_kwargs1)

In [ ]:
training_args1 = training_args()
model_input_args1 = model_input_args()
training_args1['to_plot']=True

train_res = json.loads(som_wrapper.train(**{**model_input_args1,**training_args1},json_data=json_data))

eval_args1 = eval_args()
print(train_res)
eval_args1['model_path'] = train_res['models'][0][assetno[0]]
test_res = som_wrapper.evaluate(**eval_args1,json_data=json_data)
print(test_res)

## Test 8:
#### Testing No Data exception :
* To get empty dataframe we set from and to timestamp to be not in range of the timestamps in dataset analysed

## Test 9:
#### Testing Database connectivity  exception :
* To test this we edit the db properties and run the algo

In [ ]:
writer_configs.table_name = 'f'

In [ ]:
db_properties.db_connection

### We change the db name and we expect a database error as shown below

In [ ]:
db_properties.db_connection['dbname'] = 'eg'

### Now we try to change the password of db properties and observe the exception handled

In [ ]:
db_properties.db_connection['password']='fef'

In [ ]:
%%writefile_run db_properties.py

db_connection = {'dbname': 'Cerebra',
 'host': '127.0.0.1',
 'password': 'givemeachance',
 'port': '5432',
 'user': 'postgres'}

### Now we change the table name in which we are writing, and we observe that relation doesn't exist as seen below

In [ ]:
writer_configs.table_name = 'ffee'

In [ ]:
writer_configs.table_name = 'public.log_asset_timeline'

## Test 10:
#### Testing random exceptions :

## Test 11:
#### Testing High Performance :

In [ ]:
reader_kwargs1= csv_helper.get_csv_kwargs(infile='../../dataset/bearings.csv',filename='bearings.csv',has_time=False)
json_data = reader_helper.read(reader_kwargs=reader_kwargs1)
model_input_args1,training_args1,eval_args1 = get_kwargs()

In [ ]:
%%timeit -n 1 -r 1

training_args1['to_plot']=False
training_args1['epochs']= 10
training_args1['batch_size'] = 32
train_res = json.loads(som_wrapper.train(**{**model_input_args1,**training_args1},json_data=json_data))

eval_args1['anom_thres'] = 1
eval_args1['model_path'] = train_res['models'][0][assetno[0]]
eval_args1['to_plot']=False
test_res = som_wrapper.evaluate(**eval_args1,json_data=json_data,mode=som_wrapper.mode_options[1])
print(test_res)

# Conclusion:
* Hence we observe that **Bayesian Changepoint Detection** works well only on level shifts or variational shift datasets over outlier or surge,sag datasets